In [1]:
# Путь для моделей, необходмио указывать до подключения библиотек transformers
import os
os.environ["TRANSFORMERS_CACHE"] = "D:\\_HuggingFace\\models"

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import json
from Levenshtein import distance

from tqdm.auto import tqdm

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import os
# import ast
import re


C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
import gc
gc.collect()

63

In [5]:
tqdm.pandas()

# Загрузка данных

In [6]:
PATH = ''

In [7]:
# Подключение к базе данных SQLite
conn = sqlite3.connect('buhpulse_data.sqlite')
# Создание курсора
cursor = conn.cursor()
# Запрос списка таблиц
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# Получение результатов
tables = cursor.fetchall()
# Закрытие соединения с базой данных
conn.close()

In [8]:
# Подключение к базе данных SQLite
conn = sqlite3.connect('buhpulse_data.sqlite')
df = pd.DataFrame()
for i in tqdm(tables):
    # Запрос данных из таблицы
    query = f"SELECT * FROM {i[0]}"
    
    # Чтение данных в DataFrame
    df_forum = pd.read_sql_query(query, conn)
    df_forum['messages'] = i[0]
    df = pd.concat([df, df_forum])
    
conn.close()
df.shape    

  0%|          | 0/12 [00:00<?, ?it/s]

(323603, 5)

In [9]:
df['date'].min(), df['date'].max()

('2024-01-01', '2024-09-30')

## Анализируем  БухЭксперт8
Уметь находить мнения о компании «БухЭксперт8» и определять их качество (положительно, нейтрально/упоминание, отрицательно)

## Ищем все упомниннаия БухЭксперт8
Используем расстояние Левенштейна

In [10]:
# Список слов, с которыми сравниваем
word_list = ['БухЭксперт8', 'buhexpert8']
word_list = list(map(str.lower,word_list))


# Максимальное расстояние Левенштейна для слова
max_distance = 3

# Функция для проверки, содержит ли предложение похожее слово
def contains_similar_word(sentence, word_list, max_distance):
    words = sentence.split()  # Разбиваем предложение на слова
    return any(
        distance(word.lower().strip('.,!?'), target_word) <= max_distance
        for word in words
        for target_word in word_list
    )

# Применяем фильтр к датафрейму
df['contains_similar_buhexpert'] = df['text'].progress_apply(
    lambda x: contains_similar_word(x, word_list, max_distance)
)

# Дополнительно ищем четкие совпадение которые не определились через расстояние Ливенштейна и формируем список найденных 
filtered_df = df[df['contains_similar_buhexpert'] |
    ((df['text'].str.lower().str.contains('бух эксперт')) | 
    (df['text'].str.lower().str.contains('бухэксперт')) | 
    (df['text'].str.lower().str.contains('buhexpert')) | 
    (df['text'].str.lower().str.contains('buh expert'))
    )
]
# # Отбираем строки с подходящими предложениями
# filtered_df = df[df['contains_similar_buhexpert']]
filtered_df.shape

  0%|          | 0/323603 [00:00<?, ?it/s]

(164, 6)

## Анализ тональности

### Загрзука модели

In [11]:
SYSTEM_PROMPT = """
You are an assistant that always provides responses in JSON format.
Вам предоставлен текст комментария пользователя о сервисе бухгалтерского учета. 
Ваша задача — определить, является ли этот комментарий положительным или отрицательным, с разделением на 5 категорий: 
- Очень положительный – very_positive
- Положительный – positive
- Нейтральный – neutral
- Отрицательный – negative - пользователь не доволен именно сервисом, или например сервис очень дорогой
- Очень отрицательный – very_negative - пользователь не доволен именно сервисом, или не смог что-то там найти
Старайся использовать все 5 категорий.

Если пользователь ссылается на сервис с целью предоставления достоверной информации, то это считается хорошим примером для сервиса.
Если пользователь в комментариях высказывается негативно, но при этом он ссылается на сервис только как на источник инфомрации, то этот комментарий положительным.
Если пользователь в комментариях высказывается негативно, но при этом он ссылается на бухэксперт (buhexpert8) на статью или на урок или на другой материал, то этот комментарий положительным.

Также отразите долю привлекательности сервисом от 0 до 1, где 0 - очень отрицательный (very_negative), 0.5 - нейтральный (neutral), а 1 - очень положительный (very_positive).
Если пользователь ссылается на сайт или советует посмотреть сервис, тогда это положительный комментарий.

Также отразите основные причины, по которым пользователи оценивали сервис тем или иным образом (например, удобство интерфейса, поддержка клиентов, цена и т.д.).

Результат должен быть представлен в формате JSON:

Верните извлеченную информацию в следующем формате:
```{{"emotion": "positive", "emotion_score": 0.65, "reason": "Основная причина, по которой пользователь оценивал сервис"}}```

Ensure the output is valid JSON and matches the specified schema, if provided.
"""

In [12]:
model_path = 'NousResearch/Hermes-3-Llama-3.1-8B'  #название модели https://huggingface.co/
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Блок квантования
# # # Configure 8-bit quantization
# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=True, # Используется 8-битное квантован
#     bnb_8bit_quant_type='nf4',  # 'nf4'
# )

# # # # # Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False,  # Не используется 8-битное квантование
    load_in_4bit=True,   # Включаем 4-битное квантование
    bnb_4bit_quant_type='fp4',  # 4-битное квантование с плавающей точкой
)

# Load model with quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},  # Использование bfloat16 в пайплайне
    device_map="auto",                             # Автоматическое распределение модели по устройствам
    pad_token_id=tokenizer.eos_token_id,
    truncation=True,
)

# Функция для инференса LLM
def interact(
    user_message,
    system_prompt=SYSTEM_PROMPT,
    temperature=0.1,
    max_length=512
):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt")
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            temperature=temperature,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=pad_token_id
        )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
def extract_response(text:str):
    try_count = 3
    is_good_result = False
    while try_count > 0 and is_good_result == False:
        assistant_response = interact(text.strip(), SYSTEM_PROMPT)
        try_count -= 1
        try:
            json_result = json.loads(assistant_response)
            is_good_result = True
            # print(json.dumps(json_result, indent=4))  # Вывод с форматированием
        except json.JSONDecodeError as e:
            try:
                assistant_response = assistant_response.replace('```', '').replace('JSON', '').replace('json', '').strip()
                if assistant_response[-1] != '}':
                    assistant_response += '}'
                json_result = json.loads(assistant_response)
                is_good_result = True
            except json.JSONDecodeError as e:    
                try:
                    assistant_response = assistant_response.replace('```', '').replace('JSON', '').replace('json', '').replace('"','""').strip()
                    if assistant_response[-1] != '}':
                        assistant_response += '}'
                    json_result = json.loads(assistant_response)
                    is_good_result = True
                except json.JSONDecodeError as e:
                    print("Error: Invalid JSON response")
                    print(assistant_response)
    if not is_good_result:
        print('Не получилось обработать часть данных')
        return None, None, None
    return json_result["emotion"], json_result["emotion_score"], json_result["reason"]

In [15]:
# Чистим текст 
import re
def clean_text(text):
    # оставляем только слова и цифры
    pattern = r"[a-zA-Z]+|[\d]+|[а-яА-ЯЁё]+"
    text = " ".join(re.findall(pattern, text.lower()))
    return text
# Заменяем ссылки
# Регулярное выражение для поиска ссылок
url_pattern = r'https?://[^\s]+'
# Заменяем ссылки на стандартную фразу
standard_phrase = "[ссылка на БухЭксперт8]"
# updated_text = 
filtered_df['clear_text'] = filtered_df['text'].progress_apply(lambda x: clean_text(re.sub(url_pattern, standard_phrase, x)))


  0%|          | 0/164 [00:00<?, ?it/s]

C:\Users\Profit77\AppData\Local\Temp\ipykernel_7648\3635577209.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['clear_text'] = filtered_df['text'].progress_apply(lambda x: clean_text(re.sub(url_pattern, standard_phrase, x)))


In [16]:
%%time
# Производим расчет
filtered_df[['emotion', 'emotion_score', 'reason']] = filtered_df['clear_text'].progress_apply(lambda x: pd.Series(extract_response(x.strip())))

  0%|          | 0/164 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\transformers\generation\utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\transformers\models\llama\modeling_llama.py:602: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)

CPU times: total: 1h 13min 28s
Wall time: 13min 10s


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
filtered_df['emotion'].value_counts()

emotion
positive    139
negative     15
neutral      10
Name: count, dtype: int64

In [22]:
# Сохрнаяем результат
filtered_df.to_csv('emotion_buhexpert8_df_v2.csv')
filtered_df.shape

(164, 10)

In [23]:
filtered_df

,id,date,user_id,text,messages,contains_similar_buhexpert,clear_text,emotion,emotion_score,reason
4834,4835,2024-01-12,128,Это с сайта бухэксперт приведен пример,messages_2,True,это с сайта бухэксперт приведен пример,positive,0.75,Пользователь ссылается на сервис бухэксперт ка...
10825,10826,2024-01-19,31,Бухэксперт - подробно целая тема..))),messages_2,True,бухэксперт подробно целая тема,positive,0.70,Пользователь ссылается на бухгалтерский экспер...
10883,10884,2024-01-19,547,В Бухэксперте есть,messages_2,True,в бухэксперте есть,positive,0.50,Пользователь ссылается на сервис Buhexpert как...
10937,10938,2024-01-20,13,"Ну есть стаж, а там о выплатах фсс инфа, это ч...",messages_2,True,ну есть стаж а там о выплатах фсс инфа это что...,positive,0.50,"Пользователь не доволен именно сервисом, но сс..."
11171,11172,2024-01-20,13,Если вам это поможет https://buhexpert8.ru/1s-...,messages_2,False,если вам это поможет ссылка на бухэксперт 8,positive,0.70,Пользователь ссылается на сервис Buhexpert 8 к...
...,...,...,...,...,...,...,...,...,...,...
9777,9778,2024-07-23,64,Здравствуйте. На сайте Бухэксперт8 есть подроб...,messages_13,True,здравствуйте на сайте бухэксперт 8 есть подроб...,positive,0.70,Пользователь ссылается на сервис бухэксперт 8 ...
11227,11228,2024-08-20,796,Долго будем Так гадать... Вот вам ссылка... Мо...,messages_13,False,долго будем так гадать вот вам ссылка может де...,negative,0.60,"Пользователь высказывается негативно, но ссыла..."
11569,11570,2024-08-28,57,Добрый) https://buhexpert8.ru/1s-buhgalteriya/...,messages_13,False,добрый ссылка на бухэксперт 8,positive,0.70,Пользователь ссылается на сервис Buhexpert 8 к...
11832,11833,2024-09-07,91,Вы быстрее и по шагам найдёте подробный ответ ...,messages_13,True,вы быстрее и по шагам найдёте подробный ответ ...,positive,0.70,Пользователь использует сервис как источник ин...
